In [ ]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.patches as patches
import sys
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [ ]:
%matplotlib inline
import geopandas as gpd
import contextily as cx

### Load region ecological metadata

In [ ]:
metadata = pd.read_csv("../../config/metadata.tsv", sep="\t")
#eco['mining'] = eco.Ecology.str.contains("mining")
#eco = eco.rename(columns={'Community ': 'location'})
#eco = eco[['District', 'location', 'Ecology', 'mining', 'IRS STATUS']]

### Read metadata and get locs

In [ ]:
# Read metadata 
metadata = pd.read_csv("../../config/metadata.tsv", sep="\t")

In [ ]:
meta_locs = metadata.groupby('location').agg({'latitude':'mean', 'longitude':'mean'}).reset_index()
meta = pd.crosstab(metadata['species_gambiae_coluzzii'], metadata['location']).T.reset_index()
meta = meta.merge(meta_locs)

In [ ]:
eco = metadata[['location', 'mining']].drop_duplicates().reset_index(drop=True)
meta = meta.merge(eco)

In [ ]:
meta = meta.merge(eco)

In [ ]:
meta.plot(x='latitude', y='longitude', kind='scatter', c='coluzzii', colormap='Greys', s=meta['coluzzii'], figsize=[15,10])

In [ ]:
# Sample positions
GaardianGeo = gpd.GeoDataFrame(
    metadata, geometry=gpd.points_from_xy(metadata.longitude, metadata.latitude, crs="EPSG:4326"))

GaardianGeo = GaardianGeo.to_crs(epsg=3857)

ax = GaardianGeo.plot(figsize=(15, 13), alpha=0.5, edgecolor='k', c=pd.factorize(GaardianGeo['species_gambiae_coluzzii'])[0])
cx.add_basemap(ax)

In [ ]:
meta_wide = meta.melt(id_vars=['location', 'mining', 'latitude', 'longitude'], value_vars=['coluzzii', 'gambiae'], var_name="species", value_name="counts")

In [ ]:
import matplotlib.patches as mpatches


In [ ]:
meta

In [ ]:
# Sample positions
GaardianGeo = gpd.GeoDataFrame(
    meta, geometry=gpd.points_from_xy(meta.longitude, meta.latitude, crs="EPSG:4326"))

GaardianGeo = GaardianGeo.to_crs(epsg=4326)

GaardianGeo.loc[:, 'total'] = GaardianGeo.loc[:, 'gambiae'] + GaardianGeo.loc[:, 'coluzzii'] 

GaardianGeo.loc[:,"coluzzii_fraction"] = GaardianGeo.loc[:,"coluzzii"]/ GaardianGeo.loc[:,'total']  
GaardianGeo.loc[:,"gambiae_fraction"] = GaardianGeo.loc[:,"gambiae"]/ GaardianGeo.loc[:,'total']  

labels = ['coluzzii', 'gambiae']
palette = sns.color_palette(['dodgerblue', 'indianred'], n_colors=len(labels))
# add a colour for other_resistant
#palette.append((0, 0, 0))
# add a colour for wt # check
sns.palplot(palette)
plt.gca().set_xticklabels(labels);

In [ ]:
subplot_kw = dict(projection=ccrs.PlateCarree())
# figsize here is the entire figure size in inches, reduced later by bbox_inches
# This dpi setting affects the display size of the figure in the notebook
fig, ax = plt.subplots(figsize=(20, 20), subplot_kw=subplot_kw)
GaardianGeo.to_crs(ccrs.PlateCarree())
GaardianGeo.plot(ax=ax)
cx.add_basemap(ax=ax, crs=ccrs.PlateCarree())

pie_scale_factor = 0.00003
for idx, row in GaardianGeo.iterrows():
    
    ratios = np.asarray([row[f"{k}_fraction"] for k in labels])
    ratios = np.append(ratios, 1 - np.sum(ratios))
    # wedgeprops is used here just to pass the zorder command
    centre = (row['longitude'], row['latitude']) #(row.geometry.x + row.offset_lon, row.geometry.y + row.offset_lat)
    radius= np.sqrt(row.total * pie_scale_factor)
    
    if row['mining'] == 'mining':
        ax.add_patch(plt.Circle(xy=centre, radius=radius, facecolor='none', edgecolor='gold', zorder=8, lw=4))
    else:
        ax.add_patch(plt.Circle(xy=centre, radius=radius, facecolor='none', edgecolor='k', zorder=8, lw=3))
    ax.pie(ratios, wedgeprops=dict(zorder=7, linewidth=0), colors=palette, center=centre, radius=radius, shadow=False)

    lbl = row['location']
    #if row.label == 'bottom':
    #    ax.text(centre[0], centre[1] + (radius + (0.1 * radius)), lbl, ha='center', va='bottom', fontsize=16, fontweight='bold', bbox=dict(edgecolor='w', facecolor='w', pad=1, alpha=0, ), zorder=6)
   # if row.label == 'top':
    ax.text(centre[0], centre[1] - (radius + (0.1 * radius)), lbl, ha='center', va='top', fontsize=16, fontweight='bold', bbox=dict(edgecolor='w', facecolor='w', pad=1, alpha=0, ), zorder=6)

   # if row.offset_lat != 0 or row.offset_lon != 0:
   #     ax.plot([row.geometry.x, row.geometry.x + row.offset_lon],[row.geometry.y, row.geometry.y + row.offset_lat], 'k-', lw=2, zorder=5)
    
ax.set_extent([-2.1, -1.2, 5.9, 6.45])

# where some data has already been plotted to ax
handles, labels = ax.get_legend_handles_labels()
# manually define a new patch 
patch = mpatches.Patch(color='dodgerblue', label='An. coluzzii')
# handles is a list, so append manual patch
handles.append(patch) 
patch = mpatches.Patch(color='indianred', label='An. gambiae')
# handles is a list, so append manual patch
handles.append(patch) 
patch = mpatches.Patch(color='gold', label='Informal Gold mining')
# handles is a list, so append manual patch
handles.append(patch)
plt.legend(handles=handles, loc='upper left',fontsize=16)


In [ ]:
def makegpd(meta):

    # Sample positions
    GaardianGeo = gpd.GeoDataFrame(
        meta, geometry=gpd.points_from_xy(meta.longitude, meta.latitude, crs="EPSG:4326"))

    GaardianGeo = GaardianGeo.to_crs(epsg=4326)

    GaardianGeo.loc[:, 'total'] = GaardianGeo.loc[:, 'gambiae'] + GaardianGeo.loc[:, 'coluzzii'] 

    GaardianGeo.loc[:,"coluzzii_fraction"] = GaardianGeo.loc[:,"coluzzii"]/ GaardianGeo.loc[:,'total']  
    GaardianGeo.loc[:,"gambiae_fraction"] = GaardianGeo.loc[:,"gambiae"]/ GaardianGeo.loc[:,'total']  

    labels = ['coluzzii', 'gambiae']
    palette = sns.color_palette(['dodgerblue', 'indianred'], n_colors=len(labels))
    # add a colour for other_resistant
    #palette.append((0, 0, 0))
    # add a colour for wt # check
    return(GaardianGeo, labels, palette)

def plotSampleMap(GaardianGeo):
    subplot_kw = dict(projection=ccrs.PlateCarree())
    # figsize here is the entire figure size in inches, reduced later by bbox_inches
    # This dpi setting affects the display size of the figure in the notebook
    fig, ax = plt.subplots(figsize=(20, 20), subplot_kw=subplot_kw)
    GaardianGeo.to_crs(ccrs.PlateCarree())
    GaardianGeo.plot(ax=ax)
    cx.add_basemap(ax=ax, crs=ccrs.PlateCarree())

    pie_scale_factor = 0.00003
    for idx, row in GaardianGeo.iterrows():
        
        ratios = np.asarray([row[f"{k}_fraction"] for k in labels])
        ratios = np.append(ratios, 1 - np.sum(ratios))
        # wedgeprops is used here just to pass the zorder command
        centre = (row['longitude'], row['latitude']) #(row.geometry.x + row.offset_lon, row.geometry.y + row.offset_lat)
        radius= np.sqrt(row.total * pie_scale_factor)
        
        if row['mining'] == 'mining':
            ax.add_patch(plt.Circle(xy=centre, radius=radius, facecolor='none', edgecolor='gold', zorder=8, lw=4))
        else:
            ax.add_patch(plt.Circle(xy=centre, radius=radius, facecolor='none', edgecolor='k', zorder=8, lw=3))
        ax.pie(ratios, wedgeprops=dict(zorder=7, linewidth=0), colors=palette, center=centre, radius=radius, shadow=False)

        lbl = row['location']
        #if row.label == 'bottom':
        #    ax.text(centre[0], centre[1] + (radius + (0.1 * radius)), lbl, ha='center', va='bottom', fontsize=16, fontweight='bold', bbox=dict(edgecolor='w', facecolor='w', pad=1, alpha=0, ), zorder=6)
    # if row.label == 'top':
        ax.text(centre[0], centre[1] - (radius + (0.1 * radius)), lbl, ha='center', va='top', fontsize=16, fontweight='bold', bbox=dict(edgecolor='w', facecolor='w', pad=1, alpha=0, ), zorder=6)

    # if row.offset_lat != 0 or row.offset_lon != 0:
    #     ax.plot([row.geometry.x, row.geometry.x + row.offset_lon],[row.geometry.y, row.geometry.y + row.offset_lat], 'k-', lw=2, zorder=5)
        
    ax.set_extent([-2.1, -1.2, 5.9, 6.45])

    # where some data has already been plotted to ax
    handles, labels = ax.get_legend_handles_labels()
    # manually define a new patch 
    patch = mpatches.Patch(color='dodgerblue', label='An. coluzzii')
    # handles is a list, so append manual patch
    handles.append(patch) 
    patch = mpatches.Patch(color='indianred', label='An. gambiae')
    # handles is a list, so append manual patch
    handles.append(patch) 
    patch = mpatches.Patch(color='gold', label='Informal Gold mining')
    # handles is a list, so append manual patch
    handles.append(patch)
    plt.legend(handles=handles, loc='upper left',fontsize=16)
return(fig, ax)

In [ ]:
fst =pd.read_csv("../../results/Fst_replicates.tsv")